In [49]:
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np


In [50]:
df = pd.read_csv("citybikes_all_2016_2023.csv")
'''
failed = []
for u in df["Station id"].unique():
    try:
        int(u)
    except:
        failed.append(u)
df["Station id"] = df["Station id"].map({"*71":71,"*88":88}).fillna(df["Station id"])
print(df["Station id"].unique())
'''


In [52]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18550810 entries, 0 to 18550809
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Departure             object 
 1   Return                object 
 2   Departure station id  int64  
 3   Return station id     int64  
 4   Covered distance (m)  float64
 5   Duration (sec.)       float64
 6   Departure Weekday     int64  
 7   Return Weekday        int64  
 8   Departure Hour        int64  
 9   Departure Month       int64  
 10  Return Hour           int64  
 11  Return Month          int64  
 12  Year                  int64  
 13  Departure Day         int64  
 14  Return Day            int64  
dtypes: float64(2), int64(11), object(2)
memory usage: 2.1+ GB
None


In [53]:
print(df.info())
failed = []
'''
for u in df["Return station id"].unique():
    try:
        int(u)
    except:
        failed.append(u)
print(failed)
'''
df["Return station id"] = df["Return station id"].map({"*71":71,"*88":88}).fillna(df["Return station id"])

df["Departure station id"] = df["Departure station id"].map({"*71":71,"*88":88}).fillna(df["Departure station id"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18550810 entries, 0 to 18550809
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Departure             object 
 1   Return                object 
 2   Departure station id  int64  
 3   Return station id     int64  
 4   Covered distance (m)  float64
 5   Duration (sec.)       float64
 6   Departure Weekday     int64  
 7   Return Weekday        int64  
 8   Departure Hour        int64  
 9   Departure Month       int64  
 10  Return Hour           int64  
 11  Return Month          int64  
 12  Year                  int64  
 13  Departure Day         int64  
 14  Return Day            int64  
dtypes: float64(2), int64(11), object(2)
memory usage: 2.1+ GB
None


In [54]:
def to_int(itm):
    try:
        return int(itm)
    except:
        return np.nan

df["Departure station id"] = df["Departure station id"].apply(to_int)

df["Return station id"] = df["Return station id"].apply(to_int)
print(df.isna().sum())


Departure               0
Return                  0
Departure station id    0
Return station id       0
Covered distance (m)    0
Duration (sec.)         0
Departure Weekday       0
Return Weekday          0
Departure Hour          0
Departure Month         0
Return Hour             0
Return Month            0
Year                    0
Departure Day           0
Return Day              0
dtype: int64


In [21]:
print(df.shape)
df.dropna(inplace=True)
print(df.shape)
print(df.isna().sum())


(19732128, 17)
(19400926, 17)
Departure                 0
Return                    0
Departure station id      0
Departure station name    0
Return station id         0
Return station name       0
Covered distance (m)      0
Duration (sec.)           0
Departure Weekday         0
Return Weekday            0
Departure Hour            0
Departure Month           0
Return Hour               0
Return Month              0
Year                      0
Departure Day             0
Return Day                0
dtype: int64


In [24]:
df.drop(["Departure station name","Return station name"], axis=1, inplace=True)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 19400926 entries, 64620 to 19732127
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Departure             object 
 1   Return                object 
 2   Departure station id  float64
 3   Return station id     float64
 4   Covered distance (m)  float64
 5   Duration (sec.)       float64
 6   Departure Weekday     int64  
 7   Return Weekday        int64  
 8   Departure Hour        int64  
 9   Departure Month       int64  
 10  Return Hour           int64  
 11  Return Month          int64  
 12  Year                  int64  
 13  Departure Day         int64  
 14  Return Day            int64  
dtypes: float64(4), int64(9), object(2)
memory usage: 2.3+ GB
None


In [27]:
df["Departure station id"] = df["Departure station id"].astype(int)
df["Return station id"] = df["Return station id"].astype(int)

print(df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 19400926 entries, 64620 to 19732127
Data columns (total 15 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Departure             object 
 1   Return                object 
 2   Departure station id  int64  
 3   Return station id     int64  
 4   Covered distance (m)  float64
 5   Duration (sec.)       float64
 6   Departure Weekday     int64  
 7   Return Weekday        int64  
 8   Departure Hour        int64  
 9   Departure Month       int64  
 10  Return Hour           int64  
 11  Return Month          int64  
 12  Year                  int64  
 13  Departure Day         int64  
 14  Return Day            int64  
dtypes: float64(2), int64(11), object(2)
memory usage: 2.3+ GB
None


In [55]:
temp_dep = df.groupby(["Departure station id","Year", "Departure Month", "Departure Day", "Departure Hour", "Departure Weekday"])["Covered distance (m)"].count().reset_index(name="Departure count")
temp_ret = df.groupby(["Return station id","Year", "Return Month", "Return Day", "Return Hour", "Return Weekday"])["Covered distance (m)"].count().reset_index(name="Return count")


In [56]:
train_df = pd.merge(temp_dep, temp_ret, 
                     left_on=['Departure station id', 'Departure Month', 'Departure Day', 'Departure Weekday', 'Departure Hour'],
                     right_on=['Return station id', 'Return Month', 'Return Day', 'Return Weekday', 'Return Hour'],
                     suffixes=('_departure', '_return'),
                     how='outer')


In [57]:
train_df = train_df.drop(['Year_return','Return station id', 'Return Month', 'Return Day', 'Return Weekday', 'Return Hour'], axis=1)
train_df = train_df.rename(columns={'Departure station id': 'Station id',
                                                'Departure Month': 'Month',
                                                'Departure Day': 'Day',
                                                'Departure Weekday': 'Weekday',
                                                'Departure Hour': 'Hour',
                                                'Year_departure':'Year'})
train_df.fillna(0, inplace=True)
train_df.head()


,Station id,Year,Month,Day,Hour,Weekday,Departure count,Return count
0,1.0,2017.0,5.0,2.0,9.0,1.0,3.0,0.0
1,1.0,2023.0,5.0,2.0,9.0,1.0,4.0,0.0
2,1.0,2017.0,5.0,2.0,10.0,1.0,1.0,2.0
3,1.0,2017.0,5.0,2.0,12.0,1.0,1.0,2.0
4,1.0,2017.0,5.0,2.0,12.0,1.0,1.0,2.0


In [34]:
train_df = train_df.drop(['Year_return'], axis=1)
train_df = train_df.rename(columns={'Year_departure':'Year'})


In [35]:
train_df.head()


,Station id,Year,Month,Day,Hour,Weekday,Departure count,Return count
0,1.0,2017.0,5.0,2.0,9.0,1.0,3.0,0.0
1,1.0,2023.0,5.0,2.0,9.0,1.0,4.0,0.0
2,1.0,2017.0,5.0,2.0,10.0,1.0,1.0,2.0
3,1.0,2017.0,5.0,2.0,12.0,1.0,1.0,2.0
4,1.0,2017.0,5.0,2.0,12.0,1.0,1.0,2.0


In [58]:
#df[["Departure Count", "Return Count"]] = df[["Departure Count", "Return Count"]].astype(int)
X = train_df[["Year","Station id","Month", "Day", "Weekday", "Hour"]]
y = train_df[["Departure count", "Return count"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


In [26]:
model = RandomForestRegressor(n_estimators=25, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


In [65]:
import xgboost as xgb
xbgr = xgb.XGBRegressor(n_estimators=2000).fit(X_train,y_train)
y_pred2 = xbgr.predict(X_test)


In [60]:
import lightgbm as lgbm
from sklearn.multioutput import MultiOutputRegressor
lgbmr = MultiOutputRegressor(lgbm.LGBMRegressor(n_estimators=4000)).fit(X_train,y_train)
y_pred3 = lgbmr.predict(X_test)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.072340 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 332
[LightGBM] [Info] Number of data points in the train set: 6616269, number of used features: 6
[LightGBM] [Info] Start training from score 2.828513
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.060117 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 332
[LightGBM] [Info] Number of data points in the train set: 6616269, number of used features: 6
[LightGBM] [Info] Start training from score 2.828735


In [61]:
dummy = DummyRegressor(strategy="mean")
dummy.fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)


In [67]:
import optuna
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error

model_names = ["y_pred2", "y_pred3"]

def objective(trial):
    weights = {name : trial.suggest_int(name,1,100,step=3) for name in model_names}
    probs = []
    probabilities = {name : (weights[name])*eval(name)/sum(weights.values()) for name in model_names}
    for i in range(len(probabilities["y_pred2"])):
        probs.append(sum([probabilities[name][i] for name in model_names]))
        #print(f"prob {i} ok")
    #print("Probs: ",np.array(probs))
    mae = mean_absolute_error(y_test, np.array(probs))
    #print("All ok")
    return mae

study = optuna.create_study(
    direction   = "minimize",
    sampler     = TPESampler(seed=42),
)
study.optimize(objective, n_trials=100)
print("Best R2", study.best_value)
print("Best params:", study.best_trial.params)


[I 2024-10-12 14:02:16,915] A new study created in memory with name: no-name-2e808d4b-03bd-42a6-ad5a-ed011b755a6b
[I 2024-10-12 14:02:19,448] Trial 0 finished with value: 1.2725445610725967 and parameters: {'y_pred2': 37, 'y_pred3': 97}. Best is trial 0 with value: 1.2725445610725967.
[I 2024-10-12 14:02:21,914] Trial 1 finished with value: 1.2764987006038055 and parameters: {'y_pred2': 73, 'y_pred3': 61}. Best is trial 0 with value: 1.2725445610725967.
[I 2024-10-12 14:02:24,375] Trial 2 finished with value: 1.2752159507453966 and parameters: {'y_pred2': 16, 'y_pred3': 16}. Best is trial 0 with value: 1.2725445610725967.
[I 2024-10-12 14:02:26,904] Trial 3 finished with value: 1.2767501332214788 and parameters: {'y_pred2': 4, 'y_pred3': 88}. Best is trial 0 with value: 1.2725445610725967.
[I 2024-10-12 14:02:29,342] Trial 4 finished with value: 1.2741762372058822 and parameters: {'y_pred2': 61, 'y_pred3': 73}. Best is trial 0 with value: 1.2725445610725967.
[I 2024-10-12 14:02:31,790]

Best R2 1.2725215546266706
Best params: {'y_pred2': 37, 'y_pred3': 88}


In [66]:
#RF
print("model:   R2:                     MAE:")
#print("RF      ",r2_score(y_test, y_pred),"   ", mean_absolute_error(y_test,y_pred))
#XBG
print("XGB     ",r2_score(y_test, y_pred2),"    ", mean_absolute_error(y_test,y_pred2))#, r2_score(y_test["Return Count"], y_predr))
#LGBM
print("LGBM    ",r2_score(y_test, y_pred3),"    ", mean_absolute_error(y_test,y_pred3))

print("DUMMY   ",r2_score(y_test, dummy_pred)," ", mean_absolute_error(y_test,dummy_pred))
#print(len(y["Return Count"].unique()))


model:   R2:                     MAE:
XGB      0.6873897121414434      1.3031354771098738
LGBM     0.7030469142366189      1.278363546017562
DUMMY    -2.5548121451723915e-07   2.4360178268939032


In [46]:
import pickle
#with open("RF.pkl",mode="wb") as fl:
#    pickle.dump(model, file=fl)
with open("XGBR_new.pkl",mode="wb") as fl:
    pickle.dump(xbgr, file=fl)
with open("LGBMR_new.pkl",mode="wb") as fl:
    pickle.dump(lgbmr, file=fl) 
